<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import glob
import datetime

# keras imports
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.models import Model

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import json
import time

In [2]:
config={
  "model"           : "ResNet50",
  "weights"         : "imagenet",
  "include_top"     : False,

  "train_path"      : ["train_p/Bonnet","train_p/Rear","train_p/Left","train_p/Right"],
  "test_path"       : "test_p",
  "features_path"   : "car_parts_check/features.h5",
  "labels_path"     : "car_parts_check/labels.h5",
  "results"         : "car_parts_check/results.txt",
  "classifier_path" : "car_parts_check/classifier.pickle",
  "model_path"      : "car_parts_check/model",

  "test_size"       : 0.20,
  "seed"            : 9,
  "num_classes"     : 4
}

In [3]:
model_name  = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]

train_path    = config["train_path"]

test_path    = config["test_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]

In [4]:
if model_name == "ResNet50":
    base_model = ResNet50(weights=weights)
    model = Model(base_model.input,base_model.output)
    image_size = (224, 224)

    print ("Successfully loaded base model and model...")

Successfully loaded base model and model...


In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 25,636,712
Trainable params: 25,583,592
Non-trainable params: 53,120
__________________________________________________________________________________________________


In [6]:
for layer in base_model.layers:
    print(layer.name)

input_1
conv1_pad
conv1_conv
conv1_bn
conv1_relu
pool1_pad
pool1_pool
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_0_bn
conv2_block1_3_bn
conv2_block1_add
conv2_block1_out
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_2_bn
conv2_block2_2_relu
conv2_block2_3_conv
conv2_block2_3_bn
conv2_block2_add
conv2_block2_out
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_2_bn
conv2_block3_2_relu
conv2_block3_3_conv
conv2_block3_3_bn
conv2_block3_add
conv2_block3_out
conv3_block1_1_conv
conv3_block1_1_bn
conv3_block1_1_relu
conv3_block1_2_conv
conv3_block1_2_bn
conv3_block1_2_relu
conv3_block1_0_conv
conv3_block1_3_conv
conv3_block1_0_bn
conv3_block1_3_bn
conv3_block1_add
conv3_block1_out
conv3_block2_1_conv
conv3_block2_1_bn
conv3_block2_1_relu
conv3_block2_2_conv
conv3_block2_2_bn
conv3_

In [7]:
features = []
labels = []
le_labels   = []
j = 1
for k in train_path:
    train_labels = os.listdir(k)
    # encode the labels
    print ("Encoding labels...")
    le = LabelEncoder()
    le.fit([tl for tl in train_labels])

    # loop over all the labels in the folder
    count = 1
    for i, label in enumerate(train_labels):
        cur_path = k + "/" + label
        for image_path in glob.glob(cur_path):
            img = image.load_img(image_path, target_size=image_size)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            feature = model.predict(x)
            flat = feature.flatten()
            features.append(flat)
            labels.append(label)
            print ("Processed - " + str(count))
            count += 1
        print ("Completed label - " + label)
    # encode the labels using LabelEncoder
    le = LabelEncoder()
    #le_labels = le.fit_transform(labels)
    l = [j for i in range(1,count)]
    j += 1
    le_label = np.array(l)
    print(le_label)
    # get the shape of training labels
    print ("Training labels: {}".format(le_label))
    print ("Training labels shape: {}".format(le_label.shape))
    le_labels.append(le_label)

Encoding labels...
Processed - 1
Completed label - bonnet (1).jpg
Processed - 2
Completed label - bonnet (10).jpg
Processed - 3
Completed label - bonnet (11).jpg
Processed - 4
Completed label - bonnet (12).jpg
Processed - 5
Completed label - bonnet (13).jpg
Processed - 6
Completed label - bonnet (14).jpg
Processed - 7
Completed label - bonnet (15).jpg
Processed - 8
Completed label - bonnet (16).jpg
Processed - 9
Completed label - bonnet (17).jpg
Processed - 10
Completed label - bonnet (18).jpg
Processed - 11
Completed label - bonnet (19).jpg
Processed - 12
Completed label - bonnet (2).jpg
Processed - 13
Completed label - bonnet (20).jpg
Processed - 14
Completed label - bonnet (21).jpg
Processed - 15
Completed label - bonnet (22).jpg
Processed - 16
Completed label - bonnet (3).jpg
Processed - 17
Completed label - bonnet (4).jpg
Processed - 18
Completed label - bonnet (5).jpg
Processed - 19
Completed label - bonnet (6).jpg
Processed - 20
Completed label - bonnet (7).jpg
Processed - 21
Co

In [8]:
le_labels = np.concatenate(le_labels).ravel()
le_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4])

In [11]:
features

[array([1.41173423e-05, 3.78989239e-06, 3.63278741e-06, 9.18444675e-06,
        6.47859588e-06, 5.61890283e-06, 3.55887596e-06, 3.29777185e-06,
        4.73122554e-06, 1.46662296e-06, 2.37820095e-05, 2.91183960e-06,
        9.94199308e-06, 5.52281836e-06, 3.02246781e-06, 4.35320044e-06,
        1.65785548e-06, 1.67085500e-06, 3.04320019e-06, 3.68727451e-05,
        2.50849462e-06, 9.83674568e-07, 5.24776055e-07, 5.94173889e-06,
        2.93226407e-07, 7.77991147e-07, 6.59183343e-07, 3.08814265e-07,
        2.05854894e-06, 3.39166945e-06, 1.63897073e-06, 2.65906647e-06,
        9.56321401e-07, 9.70186989e-07, 5.60666649e-06, 3.92835233e-07,
        8.97967493e-06, 6.40249198e-07, 1.96265592e-06, 4.84865495e-06,
        3.38930909e-06, 2.41725934e-06, 7.05779598e-07, 3.33301784e-07,
        1.99638475e-06, 1.44277988e-06, 8.88514933e-07, 1.51154302e-06,
        1.10714427e-06, 1.21689754e-06, 2.02755729e-07, 8.09775520e-05,
        3.09718274e-07, 2.95938980e-06, 7.89121316e-07, 2.144375

In [9]:
#Bonnet - 1
#Rear - 2
#Left - 3
#Right - 4

In [10]:
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("Saved model and weights to disk..")

print ("Features and labels saved..")

# end time
end = time.time()
print ("End time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

KeyError: 'input_1_ib-0'

In [16]:
config={
  "model"           : "ResNet50",
  "weights"         : "imagenet",
  "features_path"   : "car_parts_check/features.h5",
  "labels_path"     : "car_parts_check/labels.h5",
  "classifier_path" : "car_parts_check/classifier.pickle",
  "model_path"      : "cdd/car_parts_check/model",

  "test_size"       : 0.10,
  "seed"            : 9,
}

In [17]:
test_size  = config["test_size"]
seed = config["seed"]
features_path = config["features_path"]
labels_path = config["labels_path"]
classifier_path = config["classifier_path"]

In [18]:
# import features and labels
h5f_data  = h5py.File(features_path, 'r')
h5f_label = h5py.File(labels_path, 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']

features = np.array(features_string)
labels   = np.array(labels_string)

h5f_data.close()
h5f_label.close()

In [19]:
features.shape

(70, 1000)

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# verify the shape of features and labels
print(features.shape)
print(labels.shape)

print ("[INFO] training started...")
# split the training and testing data
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(features),
                                                                  np.array(labels),
                                                                  test_size=test_size,
                                                                  random_state=seed)

print ("[INFO] splitted train and test data...")
print ("[INFO] train data  : {}".format(trainData.shape))
print ("[INFO] test data   : {}".format(testData.shape))
print ("[INFO] train labels: {}".format(trainLabels.shape))
print ("[INFO] test labels : {}".format(testLabels.shape))

(70, 1000)
(70,)
[INFO] training started...
[INFO] splitted train and test data...
[INFO] train data  : (63, 1000)
[INFO] test data   : (7, 1000)
[INFO] train labels: (63,)
[INFO] test labels : (7,)


In [21]:
from xgboost import XGBClassifier

In [22]:
clf = XGBClassifier()
clf.fit(trainData, trainLabels)

C:\Users\jroha\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:13:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
preds = clf.predict(testData)
print(preds)

[3 1 1 1 2 1 2]


In [24]:
import pickle
print ("[INFO] saving model...")
pickle.dump(clf, open(classifier_path, 'wb'))

import pickle
print ("[INFO] Loading the classifier...")
classifier = pickle.load(open(config["classifier_path"], 'rb'))

[INFO] saving model...
[INFO] Loading the classifier...


In [25]:
test_labels = os.listdir(test_path)
# variables to hold features and labels
features_new = []

# loop over all the labels in the folder
count = 1
for i, label in enumerate(test_labels):
    cur_path = test_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features_new.append(flat)
        print ("Processed - " + str(count))
        count += 1
    print ("Completed label - " + label)

Processed - 1
Completed label - test (1).jpg
Processed - 1
Completed label - test (2).jpg
Processed - 1
Completed label - test (3).jpg
Processed - 1
Completed label - test (4).jpg
Processed - 1
Completed label - test (5).jpg
Processed - 1
Completed label - test (6).jpg
Processed - 1
Completed label - test (7).jpg
Processed - 1
Completed label - test (8).jpg


In [26]:
preds = clf.predict(np.array(features_new))
preds

array([4, 4, 4, 1, 1, 3, 1, 1])

In [ ]:
#Bonnet - 1
#Rear - 2
#Left - 3
#Right - 4